Prepare Database

In [1]:
#General
import os
import time
import csv

#Database Connection
import mysql.connector

#Web Scraping
import requests
from bs4 import BeautifulSoup

In [2]:
#Connect to the database
db = mysql.connector.connect(
    host="localhost",
    user="user",
    password="password",
    port="3306",
    database="realestatepredictor"
)

#Check if the table homegate exists. Set up the table if it does not exist
mycursor = db.cursor()
mycursor.execute("SHOW TABLES")
tables = mycursor.fetchall()
if ('homegate',) in tables:
    print("Table homegate exists")
else:
    print("Table homegate does not exist. Setting up table.")
    mycursor.execute("CREATE TABLE homegate (id INT AUTO_INCREMENT PRIMARY KEY, title VARCHAR(255), price FLOAT, rooms FLOAT, floor INT, address VARCHAR(255), url VARCHAR(255))")

Table homegate does not exist. Setting up table.


Homegate

In [6]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

locations = ['zurich', 'geneva', 'lausanne']  # Example locations
base_url = 'https://www.homegate.ch/rent/real-estate/city-{location}/matching-list?ep={page}'

def scrape_location(location):
    page = 0
    results = []

    while True:
        url = base_url.format(location=location, page=page)
        response = requests.get(url, headers=headers)

        if response.status_code != 200:
            break

        soup = BeautifulSoup(response.content, 'html.parser')
        listings = soup.find_all('div', class_='listing-item')  # Update this selector based on actual page structure

        if not listings:
            break

        for listing in listings:
            # Extract data from each listing (update selectors based on actual page structure):
            title = listing.find('h3', class_='listing-title').text.strip()
            price = listing.find('div', class_='listing-price').text.strip()
            # ... extract other data fields

            result = {
                'Title': title,
                'Price': price,
                # ... other data fields
            }
            results.append(result)

        page += 1
        time.sleep(1)  # Delay to avoid getting blocked

    return results

def main():
    all_results = []
    for location in locations:
        location_results = scrape_location(location)
        all_results.extend(location_results)

    if not all_results:
        print("No results found.")
        return

    # Save results to CSV
    keys = all_results[0].keys()
    with open('homegate_listings.csv', 'w', newline='', encoding='utf-8') as output_file:
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(all_results)

    print(f"{len(all_results)} listings were successfully scraped and saved to the homegate_listings.csv file.")

if __name__ == "__main__":
    main()

No results found.
